### This note book is for converting BioCXML format to IOB2
##### salma, 9th of april
##### Modular python code is written as BioCXML_to_IOB2_coversion_tool.py

In [4]:
import collections
import csv
import sys
import codecs
from xml.dom.minidom import parse
import xml.dom.minidom
from bs4 import BeautifulSoup


with codecs.open("../corona/manuscript/Supplemental_file4.xml", "r", "utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")
print(soup)








<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE collection SYSTEM "BioC.dtd">

<collection>
<source/>Pubmed
  <date>20200306</date>
<key></key>
<infon key="BQ_URL">http://www.bioqrator.org</infon>
<infon key="BQ_NAME">Corona_gold</infon>
<document>
<id>31991541</id>
<infon key="BQ_FROM">PubMed</infon>
<infon key="BQ_PIESCORE">-0.4251</infon>
<infon key="BQ_DONE">NO</infon>
<infon key="BQ_CURATABLE">YES</infon>
<passage>
<infon key="type">title</infon>
<offset>0</offset>
<text>Return of the Coronavirus: 2019-nCoV. </text>
<annotation id="A4">
<infon key="representative_name">SARS-CoV-2</infon>
<infon key="type">Virus_SARS-CoV-2</infon>
<infon key="NCBI:txid">2697049</infon>
<location length="9" offset="27"></location>
<text>2019-nCoV</text>
</annotation>
<annotation id="A5">
<infon key="representative_name">coronavirus</infon>
<infon key="type">Virus_family</infon>
<infon key="NCBI:txid">693995</infon>
<location length="11" offset="14"></location>
<text>Coronavirus</text>
</annotation>

An IOB2 format all the tokens must be tagged with "B", "I", or "O". If the tokens are name entities they could be B or I, else O. All the punctuation signs are tagged by "O".

In [3]:
IOB2_list =[]
passages = soup.find_all('passage')
### First we need to find all passages in the xml file which annotations are inside a "passage" block
for i in passages:
    ### Because I used split for spliting all the tokens I had to add space among words and puncutation signs 
    main_text = i.find('text').text
    main_text = main_text.replace(',',' , ')
    main_text = main_text.replace(':',' : ')
    main_text = main_text.replace(';',' ; ')
    main_text = main_text.replace('(',' ( ')
    main_text = main_text.replace(')',' ) ')
    main_text = main_text.replace('.',' . ')
    main_text = main_text.replace('%',' % ')
    main_text = main_text.replace('[',' [ ')
    main_text = main_text.replace(']',' ] ')
    
    ##########################################################
    ### Annotations blocks are NEs that the tag is presneted in the infon block 
    tokens = main_text.split()
    annot = i.find_all('annotation')
    words_per_pass = []
    for j in annot:
        info = j.find_all('infon')
        for k in info:
            if "key=\"type\"" in str(k):
                words_per_pass.append([j.find('text').text, k.text])
                
    #######################################################
    ## How 
    tokenCtr = 0
    while tokenCtr < len(tokens):
        isFound = False
        for wpp in words_per_pass:
            words = wpp[0].split()
            tags  = wpp[1]
            if tokens[tokenCtr:tokenCtr+len(words)] == words:
                isFound = True
                for tok in tokens[tokenCtr:tokenCtr+len(words)]:
                    if tok is tokens[tokenCtr]:
                        IOB2_list.append([tok,'B-'+ wpp[1]])
                    else:
                        IOB2_list.append([tok,'I-'+ wpp[1]])
                
                tokenCtr += len(words)-1
                break
        if not isFound:
            IOB2_list.append([tokens[tokenCtr],'O'])
        tokenCtr += 1
    ########################################################            
    ## This part is writing all the tokens by "B", "I", or "O" taggs to the output          
with open("../files/Gold_std.IOB2", 'w') as IOB2_file:           
    for line in IOB2_list:
        if line[0] == '.':
            print('\n',file=IOB2_file)
            print('\n')
        else:
            print(line[0], line[1], sep=' ',file=IOB2_file)
            print(line[0], line[1], sep=' ')
            
        

Return O
of O
the O
Coronavirus B-Virus_family
: O
2019-nCoV B-Virus_SARS-CoV-2


The O
emergence O
of O
a O
novel B-Virus_SARS-CoV-2
coronavirus I-Virus_SARS-CoV-2
( O
2019-nCoV B-Virus_SARS-CoV-2
) O
has O
awakened O
the O
echoes O
of O
SARS-CoV B-Virus_other
from O
nearly O
two O
decades O
ago O


Yet O
, O
with O
technological O
advances O
and O
important O
lessons O
gained O
from O
previous O
outbreaks O
, O
perhaps O
the O
world O
is O
better O
equipped O
to O
deal O
with O
the O
most B-Virus_SARS-CoV-2
recent I-Virus_SARS-CoV-2
emergent I-Virus_SARS-CoV-2
group I-Virus_SARS-CoV-2
2B I-Virus_SARS-CoV-2
coronavirus I-Virus_SARS-CoV-2


Mast O
cells O
contribute O
to O
coronavirus-induced O
inflammation B-Symptom
: O
new O
anti-inflammatory O
strategy O


Coronavirus B-Virus_SARS-CoV-2
, O
which O
can O
cause O
respiratory B-Disease_other
syndrome I-Disease_other
, O
to O
date O
has O
affected O
over O
seventeen O
thousand O
individuals O
, O
especially O
in O
China O


Coronavirus

fever B-Symptom
, O
dizziness B-Symptom
and O
a O
cough B-Symptom


Metagenomic O
RNA O
sequencing4 O
of O
a O
sample O
of O
bronchoalveolar O
lavage O
fluid O
from O
the O
patient O
identified O
a O
new B-Virus_SARS-CoV-2
RNA I-Virus_SARS-CoV-2
virus I-Virus_SARS-CoV-2
strain I-Virus_SARS-CoV-2
from I-Virus_SARS-CoV-2
the I-Virus_SARS-CoV-2
family I-Virus_SARS-CoV-2
Coronaviridae I-Virus_SARS-CoV-2
, O
which O
is O
designated O
here O
'WH-Human B-Virus_SARS-CoV-2
1' I-Virus_SARS-CoV-2
coronavirus I-Virus_SARS-CoV-2
( O
and O
has O
also O
been O
referred O
to O
as O
'2019-nCoV' O
) O


Phylogenetic O
analysis O
of O
the O
complete O
viral O
genome O
( O
29 O
, O
903 O
nucleotides O
) O
revealed O
that O
the O
virus B-Virus_SARS-CoV-2
was O
most O
closely O
related O
( O
89 O


1 O
% O
nucleotide O
similarity O
) O
to O
a O
group O
of O
SARS-like B-Virus_family
coronaviruses I-Virus_family
( O
genus O
Betacoronavirus B-Virus_family
, O
subgenus O
Sarbecovirus B-Virus_family
) O
that O
h